In [6]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.firefox.options import Options

import numpy as np
import pandas as pd
import time

In [7]:
class Loja:
    
    VOLTAR_PAGINA_LOJA = '/html/body/jsl/div[3]/div[9]/div[8]/div/div[1]/div/div/button'
    NOME = '/html/body/jsl/div[3]/div[9]/div[8]/div/div[1]/div/div/div[2]/div[1]/div[1]/h1'
    AVALIACAO = '/html/body/jsl/div[3]/div[9]/div[8]/div/div[1]/div/div/div[2]/div[1]/div[2]/div/div[1]/span[2]/span/span/span[2]/span[1]/button'
    ENDERECO = '/html/body/jsl/div[3]/div[9]/div[8]/div/div[1]/div/div/div[10]/div/div[1]/span[3]/span[3]'
    WEBSITE = '/html/body/jsl/div[3]/div[9]/div[8]/div/div[1]/div/div/div[13]/div/div[1]/span[3]/span[3]'
    CLASS_INFO = 'widget-pane-link'
    CLASS_INFO_XPATH = "//span[@class='widget-pane-link']"
    RANKING_CLASS = 'section-star-display'
            
    def __init__(self, element, wait, driver):
        self.element = element
        self.wait = wait
        self.element.click()
        self.driver = driver
        self.dicionarioInfo = {"Nome": ""}
        while True:
            retorno = self.dados(self.NOME)
            if retorno:
                break
      
    def infoAdicional(self, xpath):
        #informacoes = self.driver.find_elements(By.XPATH, xpath)
        try:
            informacoes = WebDriverWait(self.driver, 30).until(EC.presence_of_element_located((By.XPATH, xpath)))
            informacoes = self.driver.find_elements(By.XPATH, xpath)
        except:
            print('Objeto não encontrado')
            return
        finally:
            index = 0
            informacoes = self.driver.find_elements(By.XPATH, xpath)
            try:
                for info in informacoes:
                    print(index, info.text)
                    self.dicionarioInfo[str(index)] = info.text
                    index+=1
            except:
                return
            finally:
                return
    
    def informacao(self, xpath, info):
        infor = ''
        try:
            found = WebDriverWait(self.driver, 20).until(EC.element_to_be_clickable((By.XPATH, xpath)))
            #found = self.driver.find_elements(By.XPATH, xpath)
        except:
            print("Não encontrado")
            return False
        finally:
            try:
                self.dicionarioInfo[info] = found.text
                print(found.text)
                #print(found.text)
            except:
                print('Não há texto')
                return ''
            finally:
                return infor
                
    def informacaoClass(self, classname, tipo):
        try:
            objeto = WebDriverWait(self.driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, classname)))
            objeto = self.driver.find_element_by_class_name(classname)
        except:
            print("Objeto: " + tipo + 'não encontrado')
            return
        finally:
            try:
                self.dicionarioInfo[tipo] = objeto.text
            except:
                self.dicionarioInfo[tipo] = ''
                return
            finally:
                print(self.dicionarioInfo[tipo])
                return
    
    def getDataFrame(self):
        return self.dicionarioInfo
        #A = self.dicionarioInfo.items()
        #return pd.DataFrame(np.array(A).reshape(-1,len(A)))
        #return pd.DataFrame.from_dict(self.dicionarioInfo)
        
    def dados(self,info):
        """print(self.element.find_element_by_xpath(".//span[@class='section-result-details']").text)"""
        self.informacao(self.NOME, 'Nome')
        self.informacao(self.AVALIACAO, 'Avaliação')
        self.informacaoClass(classname = self.RANKING_CLASS, tipo='Ranking')
        #self.address = self.informacao(self.ENDERECO)
        #self.website = self.informacao(self.WEBSITE)     
        self.infoAdicional(self.CLASS_INFO_XPATH)
        
        #time.sleep(5)
        #element = self.driver.find_elements_by_class_name("widget-pane-content-shim noprint")
        
        self.driver.execute_script("document.getElementsByClassName('widget-pane-content-shim noprint')[0].style.visibility='hidden'")
        try:
            voltar = WebDriverWait(self.driver, 20).until(EC.element_to_be_clickable((By.XPATH, self.VOLTAR_PAGINA_LOJA)))

        except:
            print('ERRO 05')
            try:
                voltar = WebDriverWait(self.driver, 20).until(EC.invisibility_of_element_located((By.XPATH, self.VOLTAR_PAGINA_LOJA)))
            except:
                print('ERRO 04')
            finally:
                try:
                    voltar.click()
                    return True
                except:
                    print('ERRO  03')
                    return False
        
        finally:
            try:
                voltar.click()
                return True
            except:
                print('ERRO 02')
                return False
        


In [8]:
class GoogleScrap:
    
    ORIGIN_SEARCH_BAR = '//*[@id="tsf"]/div[2]/div[1]/div[1]/div/div[2]/input'
    MAPS_SEARCH_BAR ='searchboxinput'
    MAPS_BUTTON = '/html/body/div[6]/div[2]/div[3]/div/div/div[1]/div/div/div[1]/div/div[4]/a'
    CLASS_RESULTADOS_MAPS = "section-result"
    XPATH_RESULTADOS = "/html/body/jsl/div[3]/div[9]/div[8]/div/div[1]/div/div/div[6]"
    ORIGIN = 'https://www.google.com/maps'
    PROXIMA_PAGINA = '/html/body/jsl/div[3]/div[9]/div[8]/div/div[1]/div/div/div[4]/div[2]/div/div[1]/div/button[2]'
    PROXIMA_PAGINA_XPATH = '/html/body/jsl/div[3]/div[9]/div[8]/div/div[1]/div/div/div[4]/div[2]/div/div[1]/div/button[2]'
    PROXIMA_PAGINA_CLASSE = "n7lv7yjyC35__section-pagination-button-next"
    PROXIMA_PAGINA_ID = 'n7lv7yjyC35__section-pagination-button-next'
    
    def __init__(self, keyword, setor):
        data = {}
        self.tabela = pd.DataFrame(data)
        self.keyword = keyword
        self.setor = setor
        options = Options()
        options.headless = True
        self.driver = webdriver.Firefox(options=options, executable_path=r'./geckodriver')
        self.driver.implicitly_wait(20)
        self.wait = WebDriverWait(self.driver, 100)
        self.driver.get(self.ORIGIN)
    
    def close(self):
        self.driver.close()
        
    def pesquisarOrigin(self):
        originBar = self.driver.find_element_by_id(self.MAPS_SEARCH_BAR)
        originBar.send_keys(self.keyword)
        originBar.send_keys(Keys.ENTER)
        self.acessarSetor()       
    
    def acessarSetor(self):
        #self.wait.until(EC.element_to_be_clickable((By.XPATH ,self.MAPS_BUTTON))).click()
        print('achei')
        while True:
            
            proximo = 0
            proximo = WebDriverWait(self.driver, 20).until(EC.element_to_be_clickable((By.CLASS_NAME, self.CLASS_RESULTADOS_MAPS)))
            print(proximo)
            if not proximo:
                break
            retornou = False
            retornou = self.listarAchados()
            print(retornou)
            try:
               WebDriverWait(self.driver, 20).until(EC.element_to_be_clickable((By.XPATH, self.PROXIMA_PAGINA_XPATH)))
            except:
                print('loop finalizado')
                fim = True
            finally:
                print('loop reiniciado')
                #proximo = self.wait.until(EC.element_to_be_clickable((By.XPATH, self.PROXIMA_PAGINA_XPATH)))
                try:
                    proximo = self.driver.find_element_by_id(self.PROXIMA_PAGINA_ID)
                except:
                    print('Nao achado pelo xpath')
                    try:
                        proximo = self.driver.find_element(By.XPATH, self.PROXIMA_PAGINA_XPATH)
                    except:
                        ('Nao encontrado pela classe')
                    finally:
                        fim = False
                finally:
                    fim = False
                try:
                    proximo = WebDriverWait(self.driver, 40).until(EC.element_to_be_clickable((By.ID, self.PROXIMA_PAGINA_ID)))
                except:
                    print('Não achei o proximo')
                finally:
                    proximo.click()
            if fim:
                break
            time.sleep(10)

    def listarAchados(self):
        try:
            self.wait.until(EC.element_to_be_clickable((By.CLASS_NAME, self.CLASS_RESULTADOS_MAPS)))
        except TimeoutException:
            print("Não encontrado")
        lista = self.driver.find_elements_by_class_name(self.CLASS_RESULTADOS_MAPS)
        #print(len(lista))
        
        for loja in range(len(lista)):
            try:
                WebDriverWait(self.driver, 30).until(EC.element_to_be_clickable((By.CLASS_NAME, self.CLASS_RESULTADOS_MAPS)))
            except:
                print('ERRO 01')
            finally:
                lojas = self.driver.find_elements_by_class_name(self.CLASS_RESULTADOS_MAPS)
                #print(len(lojas))
                #print(loja)    
                teste = Loja(element = lojas[loja], wait = self.wait, driver = self.driver)
                self.tabela = self.tabela.append(teste.getDataFrame(), ignore_index=True)
                del teste
        print(self.tabela)
        return True
    def salvarTabela(self):
        self.tabela.to_csv('concorrencia.csv', encoding='utf-8')
    

In [13]:
test = GoogleScrap(keyword='Pisos Londrina', setor='teste')
test.pesquisarOrigin()
test.salvarTabela()

achei
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="d90f86b6-6896-4d1b-b43c-e462d25c32a0", element="3d971938-b994-470b-849e-7d7277538744")>
Pisos Londrina - Paver
(19)
4,3
0 
1 
2 R. Santa Teresinha, 843 - Fraternidade, Londrina - PR, 86027-620
3 MVQ5+G3 Fraternidade, Londrina - PR
4 (43) 3339-2034
5 Adicionar website
London Pisos
(42)
4,8
0 
1 
2 R. Pará, 2011 - Centro - Quebec, Londrina - PR, 86020-400
3 MRPH+5J Quebec, Londrina - PR
4 londonpisos.com.br
5 (43) 3031-1650
Pisos Passarela
(13)
4,4
0 
1 
2 R. Dolores Peralta, 20 - Parque Waldemar Hauer, Londrina - PR, 86030-270
3 PV46+M7 Ideal, Londrina - PR
4 pisospassarela.com.br
5 (43) 3339-6013
PisoFácil (Store) (Pisos e Revestimentos)
(13)
4,4
0 
1 
2 Av. Santos Dumont, 1460 - Bairro Aeroporto, Londrina - PR, 86039-090
3 MVG5+3F Bairro Aeroporto, Londrina - PR
4 (43) 3025-4445
5 Adicionar website
PisoAqui
(50)
4,2
0 
1 
2 Av. Theodoro Victorelli, 255 - Helena, Londrina - PR, 86089-050
3 MVQ3+H8 Helena, Londrina

ElementClickInterceptedException: Message: Element <button id="n7lv7yjyC35__section-pagination-button-next" class="n7lv7yjyC35__button noprint n7lv7yjyC35__button-disabled"> is not clickable at point (367,636) because another element <div class="n7lv7yjyC35__right"> obscures it


In [15]:
test.salvarTabela()

In [25]:
data = {'Nome':[],'Avaliação':[],'Endereço':[],'Website':[],'Rede Social':[]}
tabela = pd.DataFrame(data)

In [27]:
df =test.tabela.drop_duplicates(subset ="Endereço", 
                     keep = False, inplace = False) 
df

,Nome,Avaliação,Endereço,Website,Rede Social
2,Gol Materiais Para Construção,(12),"Rod. Mabio Gonçalves Palhano, 955 - Gleba Faze...",Reivindicar esta empresa,
4,Depósito de Materiais Casa & Construção,(6),"R. Pedro Botelho de Rezende, 1160 - Jardim Bur...",(43) 3341-8117,
5,Todimo,(274),"Av. Saul Elkind, 2900 - Jardim Athenas, Londri...",(43) 3372-5000,
6,Papaes Comércio Materiais de Construção,(1),"Av. Duque de Caxias, 1980 - Centro, Londrina -...",,
7,DEPÓSITO BRASIL SUL,(4),"R. Belém, 61 - Casoni, Londrina - PR, 86026-000",(43) 3323-5942,
...,...,...,...,...,...
238,Casa do Construtor Cambé,(9),"R. Belo Horizonte, Nº 16 - Centro, Cambé - PR,...",(43) 3035-6969,
239,ICESI,(1),icesi.ueniweb.com,,
240,Telhanorte Maringá,(1.010),"Av. Colombo, 3361 - Parque Industrial Bandeira...",(44) 3355-3700,
241,Votorantim Cimentos Brasil,(1),"Rua Cap Jacy Silva Pinheiros, 800 - Prq Indl C...",(43) 3348-2525,


In [22]:
test.tabela

,Nome,Avaliação,Endereço,Website,Rede Social
0,Tijotelhas,(12),"R. XV de Fevereiro, 249 - Centro, Cornélio Pro...",(43) 3524-1055,
1,BARROS CASA - O SHOPPING DA CONSTRUÇÃO,(66),"Av. Getúlio Vargas, 265, Cornélio Procópio - P...",,
2,M Silva Material de Construção,(21),"R. Benjamin Constant, 303 - Centro, Cornélio P...",,
3,Fortcasa Materiais Para Construção,(7),"Av. Getúlio Vargas, 91, Cornélio Procópio - PR...",,
4,"Comercial Andrade Materiais para Construção, A...",(10),"R. Antônio Vilas Bôas, 700 - centro, Cornélio ...",(43) 3523-7538,
...,...,...,...,...,...
70,Revesimentos e acabamentos,,negocio.site,,
71,Tozetti Marcenaria,(1),"PR-160, 63, Cornélio Procópio - PR, 86300-000",,
72,Casa dos Parafusos,(12),"Francisco Lacerda Junior, Nº1472 - Centro, Cor...",Reivindicar esta empresa,
73,AcquaPisos Loja de Piscinas,(14),"R. Francisco Morato, 225 - Centro, Cornélio Pr...",(43) 3524-5546,


In [28]:
df.to_csv('test.csv', encoding='utf-8')